In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [6]:
data = pd.read_csv('Churn_Modelling.csv')

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geography = OneHotEncoder(drop='first', sparse_output=False)
geography_encoded = onehot_encoder_geography.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geography_encoded, columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)
with open('onehot_encoder_geography.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geography, f)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [14]:
## Define a function to create a  model and try different parameters(Keras Classifiers)

def create_model(neurons = 32, layers = 1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation = 'relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [16]:
## Create a Keras classifier
model = KerasClassifier(model = create_model, epochs = 50, batch_size = 10, verbose = 0)

In [17]:
## Define the grid search parameters

param_grid = {
    'model__neurons' : [16, 32, 64, 128],
    'model__layers': [1,2],
    'epochs' : [50, 100]
}

In [ ]:
## Perform grid search
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 3)
grid_result = grid.fit(X_train, y_train)

## print the best paramerters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))